### Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
import ultralytics
import cv2 as cv
import numpy as np
from ultralytics import solutions
from datetime import datetime

from utils import get_coordinates, crop_and_mask_image, stack_image

ultralytics.checks()

Ultralytics 8.3.74  Python-3.10.11 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i7-11700 2.50GHz)
Setup complete  (20 CPUs, 31.8 GB RAM, 365.5/476.1 GB disk)
Setup complete  (20 CPUs, 31.8 GB RAM, 365.5/476.1 GB disk)


### Read the Video File and Write the Output to AVI

- You can either read the video file directly or stream the content from an RTSP (Real-Time Streaming Protocol) source, allowing for flexible video input depending on your needs.
- We will also set up the video writer to handle the output video writing.

In [ ]:
name_model = "yolo11n_100_best"
# name_model = "yolo11m"
name_video = "alpha.mp4"
name_datetime = str(datetime.now().replace(microsecond=0)).replace("-", "").replace(":", "").replace(" ", "_")
name_result = f'{name_model}_{name_video.split(".")[0]}_{name_datetime}'

cap = cv.VideoCapture(f"../../asset/{name_video}")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (
    int(cap.get(x)) 
    for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS)
)

# Video writer
video_writer = cv.VideoWriter(f"../../asset/result/{name_result}.avi", cv.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

### Define Region Coordinates

Here, we set the coordinates for specific regions to ensure accurate object tracking and analysis within the video or stream. This helps monitor and count objects effectively in different areas.

In [ ]:
success, imx = cap.read()

# # alpha.mp4
# region_points = [(780, 225), (1136, 622)]  # original
region_points = [(120, 30), (400, 475)]   # crop
x_min, x_max = 705, 1456
y_min, y_max = 132, 665
imx_crop = imx[y_min:y_max, x_min:x_max]
polygon = np.array([(0, 0), (388, 531), (750, 189), (304, 1)])

# # delta.mp4
# region_points = [(233, 800), (1795, 800)]  # original
# region_points = [(0, 250), (1178, 250)]    # crop
# x_min, x_max = 422, 1600
# y_min, y_max = 140, 510
# imx_crop = imx[y_min:y_max, x_min:x_max]
# polygon = np.array([(0, imx_crop.shape[0]), (273, 0), (950, 0), (imx_crop.shape[1], imx_crop.shape[0])])

# # beta.mp4
# region_points = [(275, 250), (580, 250)]  # original
# region_points = [(25, 130), (330, 130)]   # crop
# x_min, x_max = 250, 600
# y_min, y_max = 120, 290
# imx_crop = imx[y_min:y_max, x_min:x_max]
# polygon = np.array([(0, imx_crop.shape[0]), (130, 0), (245, 0), (imx_crop.shape[1], imx_crop.shape[0])])

# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

Original Image

In [ ]:
success, imx = cap.read()

# Buat window dan set event callback
cv.namedWindow("Image", cv.WINDOW_NORMAL)
cv.namedWindow("Image")
cv.setMouseCallback("Image", get_coordinates)
cv.imshow("Image", imx)
cv.waitKey(0)

Mouse Hovering at: (737, 242)
Mouse Hovering at: (1301, 467)


-1

Crop Image

In [ ]:
# success, imx = cap.read()

# imx_crop = imx[y_min:y_max, x_min:x_max]
# imx_crop = crop_and_mask_image(imx, x_min, x_max, y_min, y_max, polygon)
# print(imx_crop.shape)

# cv.namedWindow("Image", cv.WINDOW_NORMAL)
# cv.namedWindow("Image")
# cv.setMouseCallback("Image", get_coordinates)
# cv.imshow("Image", imx_crop)
# cv.waitKey(0)

(700, 750, 3)
Mouse Hovering at: (154, 130)
Mouse Hovering at: (452, 415)
Mouse Hovering at: (122, 32)
Mouse Hovering at: (406, 476)


-1

### Initialize the ObjectCounter Class

- Now, let's initialize the `ObjectCounter` class to track and count objects in each frame of the video.

In [473]:
# model = YOLO("yolo11m.pt")
# path = model.export(format="onnx")

In [474]:
# Init ObjectCounter
counter = solutions.ObjectCounter(
    region=region_points,                           # Pass region points
    model=f"../../asset/result/{name_model}.pt",    # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    # classes=[1, 2, 3, 5, 6, 7, 8, 17, ],            # If you want to count specific classes i.e person and car with COCO pretrained model.
    show=True,                                      # Display the output
    show_in=True,                                   # Display in counts
    show_out=True,                                  # Display out counts
    line_width=1,                                   # Adjust the line width for bounding boxes and text display
    # font_size=0.1,
)

Ultralytics Solutions:  {'region': [(25, 130), (330, 130)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'show': True, 'model': '../../asset/result/yolo11n_100_best.pt', 'classes': [1, 2, 3, 5, 6, 7, 8, 17], 'line_width': 1, 'font_size': 0.1}


### Process Video Frames

In this step, we will process each frame of the video to detect and analyze objects. This allows for real-time tracking and counting, based on the visual data in the frames.

In [ ]:
i = 0
n_skip = 1
is_skip = True

cv.namedWindow("Ultralytics Solutions", cv.WINDOW_NORMAL)
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    
    if i == n_skip and is_skip:
        i = 0
        continue
    
    im0_original = im0.copy()
    im0 = crop_and_mask_image(im0, x_min, x_max, y_min, y_max, polygon)
    im0 = counter.count(im0)
    print(f"Classwise counts: {counter.classwise_counts}")
    print(f"Counted IDs: {counter.counted_ids}")
    
    im0 = stack_image(im0_original, im0, x_min, x_max, y_min, y_max)
    counter.display_counts(im0)
    video_writer.write(im0)
    
    i += 1

cap.release()   # Release the capture
video_writer.release()
cv.destroyAllWindows()


0: 320x640 5 motorbikes, 2 cars, 1 person, 40.6ms
Speed: 1.3ms preprocess, 40.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 motorbikes, 2 cars, 35.7ms
Speed: 1.5ms preprocess, 35.7ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 motorbikes, 2 cars, 32.9ms
Speed: 1.0ms preprocess, 32.9ms inference, 3.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 motorbikes, 2 cars, 35.0ms
Speed: 1.0ms preprocess, 35.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 motorbikes, 2 cars, 35.9ms
Speed: 1.0ms preprocess, 35.9ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 motorbikes, 2 cars, 46.4ms
Speed: 0.0ms preprocess, 46.4ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 motorbikes, 2 cars, 42.0ms
Speed: 0.0ms preprocess, 42.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 motorbikes

In [ ]:
cv.waitKey(0)
cap.release()   # Release the capture
video_writer.release()
cv.destroyAllWindows()

In [478]:
counter.classwise_counts

{'motorbike': {'IN': 120, 'OUT': 0},
 'car': {'IN': 50, 'OUT': 0},
 'person': {'IN': 0, 'OUT': 0}}

In [ ]:
# cv.imshow("img", im0_original)
# cv.waitKey(0)

Camlytics Benchmark

In [480]:
# ### manual
# bicycle		: 5
# car		    : 16
# motorbike	: 68
# bus		    : 1
# person		: 4

# ### yolo11n_crop_skip1
# bicycle		: 5
# car		    : 16
# motorbike	: 65
# bus		    : 1
# person		: 4

# ### camlytics
# exit		: 76
# ---
# pedestrian	: 70
# unknown		: 1
# vehicle		: 5

# enter		: 49
# ---
# pedestrian	: 31
# unknown		: 5
# vehicle		: 13

View Average FPS on Coral

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [30]:
def create_dataframe_from_output(output: str) -> pd.DataFrame:
    list_output = [item for item in output.split("\n") if item.startswith("Speed")]
    list_output = [
        (
            float(item.split()[1].rstrip("ms")), 
            float(item.split()[3].rstrip("ms")),
        ) 
        for item in list_output
    ]
    list_output = [(*item, sum(item)) for item in list_output]
    return pd.DataFrame(list_output, columns=["fps_track_in_ms", "fps_detection_in_ms", "fps_total_in_ms"])


def visualize_fps_over_time(df: pd.DataFrame, framework="plotly"):
    if framework not in ["seaborn", "matplotlib", "plotly"]:
        framework = "plotly"
    
    if framework == "seaborn":
        # DataFrame sudah ada: df
        # Tambahkan index sebagai urutan frame
        df["frame"] = range(1, len(df) + 1)

        # Ubah ke bentuk long agar bisa langsung dipakai seaborn
        df_melted = df.melt(id_vars="frame", 
                            value_vars=["fps_track_in_ms", "fps_detection_in_ms", "fps_total_in_ms"],
                            var_name="type", value_name="ms")

        # Set gaya plot agar tampilannya modern
        sns.set(style="whitegrid")
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=df_melted, x="frame", y="ms", hue="type", marker="o", linewidth=2.5)

        # Tambahkan sentuhan desain
        plt.title("FPS Performance Over Time", fontsize=18, fontweight="bold")
        plt.xlabel("Frame Number", fontsize=14)
        plt.ylabel("Milliseconds (ms)", fontsize=14)
        plt.legend(title="Metric Type")
        plt.tight_layout()

        plt.show()
    elif framework == "matplotlib":
        # Asumsikan df sudah ada dan bernama 'df'
        # Tambahkan kolom 'frame' sebagai urutan
        df["frame"] = range(1, len(df) + 1)

        # Setup figure dan style
        plt.style.use("ggplot")  # gaya elegan minimalis
        plt.figure(figsize=(12, 6))

        # Plot masing-masing metrik
        plt.plot(df["frame"], df["fps_track_in_ms"], label="Track", marker="o", linewidth=2.5)
        plt.plot(df["frame"], df["fps_detection_in_ms"], label="Detection", marker="s", linewidth=2.5)
        plt.plot(df["frame"], df["fps_total_in_ms"], label="Total", marker="^", linewidth=2.5)

        # Label dan desain
        plt.title("FPS Performance Over Time", fontsize=18, fontweight="bold")
        plt.xlabel("Frame Number", fontsize=14)
        plt.ylabel("Milliseconds (ms)", fontsize=14)
        plt.legend(title="Metric Type")
        plt.grid(True)
        plt.tight_layout()

        plt.show()
    elif framework == "plotly":
        # Asumsikan df sudah tersedia dan bernama 'df'
        df["frame"] = range(1, len(df) + 1)

        # Ubah ke format long agar lebih cocok untuk plotly express
        df_melted = df.melt(id_vars="frame",
                            value_vars=["fps_track_in_ms", "fps_detection_in_ms", "fps_total_in_ms"],
                            var_name="Metric", value_name="Milliseconds")

        # Buat plot interaktif
        fig = px.line(df_melted, x="frame", y="Milliseconds", color="Metric",
                    title="FPS Performance Over Time",
                    markers=True,
                    template="plotly_white",
                    labels={"frame": "Frame Number", "Milliseconds": "Milliseconds (ms)", "Metric": "Metric Type"})

        # Tambahkan styling jika diperlukan
        fig.update_layout(title_font_size=18,
                        xaxis_title_font_size=14,
                        yaxis_title_font_size=14,
                        legend_title_font_size=14,
                        font=dict(size=12))

        fig.show()
    

Inference without ROI

In [35]:
output = """
0: 1800x3200 59.3ms
Speed: 2476.6ms track, 59.3ms solution per image at shape (1, 3, 1800, 3200)

1: 1800x3200 17.2ms
Speed: 458.3ms track, 17.2ms solution per image at shape (1, 3, 1800, 3200)

2: 1800x3200 7.7ms
Speed: 284.7ms track, 7.7ms solution per image at shape (1, 3, 1800, 3200)

3: 1800x3200 5.4ms
Speed: 286.2ms track, 5.4ms solution per image at shape (1, 3, 1800, 3200)

4: 1800x3200 5.1ms
Speed: 259.3ms track, 5.1ms solution per image at shape (1, 3, 1800, 3200)

5: 1800x3200 4.6ms
Speed: 265.2ms track, 4.6ms solution per image at shape (1, 3, 1800, 3200)

6: 1800x3200 5.2ms
Speed: 252.5ms track, 5.2ms solution per image at shape (1, 3, 1800, 3200)

7: 1800x3200 4.6ms
Speed: 263.2ms track, 4.6ms solution per image at shape (1, 3, 1800, 3200)

8: 1800x3200 7.9ms
Speed: 250.5ms track, 7.9ms solution per image at shape (1, 3, 1800, 3200)

9: 1800x3200 6.5ms
Speed: 268.9ms track, 6.5ms solution per image at shape (1, 3, 1800, 3200)

10: 1800x3200 9.2ms
Speed: 247.8ms track, 9.2ms solution per image at shape (1, 3, 1800, 3200)

11: 1800x3200 7.5ms
Speed: 268.3ms track, 7.5ms solution per image at shape (1, 3, 1800, 3200)

12: 1800x3200 10.5ms
Speed: 262.8ms track, 10.5ms solution per image at shape (1, 3, 1800, 3200)

13: 1800x3200 6.1ms
Speed: 265.3ms track, 6.1ms solution per image at shape (1, 3, 1800, 3200)

14: 1800x3200 67.4ms
Speed: 247.8ms track, 67.4ms solution per image at shape (1, 3, 1800, 3200)

15: 1800x3200 6.0ms
Speed: 259.6ms track, 6.0ms solution per image at shape (1, 3, 1800, 3200)

16: 1800x3200 6.8ms
Speed: 247.1ms track, 6.8ms solution per image at shape (1, 3, 1800, 3200)

17: 1800x3200 6.3ms
Speed: 259.0ms track, 6.3ms solution per image at shape (1, 3, 1800, 3200)

18: 1800x3200 6.9ms
Speed: 246.0ms track, 6.9ms solution per image at shape (1, 3, 1800, 3200)

19: 1800x3200 6.0ms
Speed: 256.7ms track, 6.0ms solution per image at shape (1, 3, 1800, 3200)

20: 1800x3200 6.5ms
Speed: 247.8ms track, 6.5ms solution per image at shape (1, 3, 1800, 3200)

21: 1800x3200 6.9ms
Speed: 258.4ms track, 6.9ms solution per image at shape (1, 3, 1800, 3200)

22: 1800x3200 3.2ms
Speed: 241.1ms track, 3.2ms solution per image at shape (1, 3, 1800, 3200)

23: 1800x3200 7.2ms
Speed: 262.3ms track, 7.2ms solution per image at shape (1, 3, 1800, 3200)

24: 1800x3200 5.1ms
Speed: 247.8ms track, 5.1ms solution per image at shape (1, 3, 1800, 3200)

25: 1800x3200 6.0ms
Speed: 253.5ms track, 6.0ms solution per image at shape (1, 3, 1800, 3200)

26: 1800x3200 3.6ms
Speed: 242.6ms track, 3.6ms solution per image at shape (1, 3, 1800, 3200)

27: 1800x3200 6.5ms
Speed: 258.0ms track, 6.5ms solution per image at shape (1, 3, 1800, 3200)

28: 1800x3200 4.1ms
Speed: 244.2ms track, 4.1ms solution per image at shape (1, 3, 1800, 3200)

29: 1800x3200 5.4ms
Speed: 257.5ms track, 5.4ms solution per image at shape (1, 3, 1800, 3200)

30: 1800x3200 6.0ms
Speed: 242.2ms track, 6.0ms solution per image at shape (1, 3, 1800, 3200)

31: 1800x3200 6.0ms
Speed: 258.6ms track, 6.0ms solution per image at shape (1, 3, 1800, 3200)

32: 1800x3200 8.2ms
Speed: 249.1ms track, 8.2ms solution per image at shape (1, 3, 1800, 3200)

33: 1800x3200 8.6ms
Speed: 253.7ms track, 8.6ms solution per image at shape (1, 3, 1800, 3200)

34: 1800x3200 7.6ms
Speed: 240.2ms track, 7.6ms solution per image at shape (1, 3, 1800, 3200)

35: 1800x3200 8.3ms
Speed: 271.2ms track, 8.3ms solution per image at shape (1, 3, 1800, 3200)

36: 1800x3200 5.3ms
Speed: 251.6ms track, 5.3ms solution per image at shape (1, 3, 1800, 3200)

37: 1800x3200 8.2ms
Speed: 259.1ms track, 8.2ms solution per image at shape (1, 3, 1800, 3200)

38: 1800x3200 7.7ms
Speed: 244.6ms track, 7.7ms solution per image at shape (1, 3, 1800, 3200)

39: 1800x3200 10.8ms
Speed: 262.7ms track, 10.8ms solution per image at shape (1, 3, 1800, 3200)

40: 1800x3200 8.2ms
Speed: 244.8ms track, 8.2ms solution per image at shape (1, 3, 1800, 3200)

41: 1800x3200 8.6ms
Speed: 256.3ms track, 8.6ms solution per image at shape (1, 3, 1800, 3200)

42: 1800x3200 11.1ms
Speed: 247.3ms track, 11.1ms solution per image at shape (1, 3, 1800, 3200)

43: 1800x3200 4.7ms
Speed: 254.1ms track, 4.7ms solution per image at shape (1, 3, 1800, 3200)

44: 1800x3200 9.2ms
Speed: 246.3ms track, 9.2ms solution per image at shape (1, 3, 1800, 3200)

45: 1800x3200 8.1ms
Speed: 257.7ms track, 8.1ms solution per image at shape (1, 3, 1800, 3200)

46: 1800x3200 9.5ms
Speed: 243.6ms track, 9.5ms solution per image at shape (1, 3, 1800, 3200)

47: 1800x3200 7.7ms
Speed: 256.2ms track, 7.7ms solution per image at shape (1, 3, 1800, 3200)

48: 1800x3200 8.5ms
Speed: 246.7ms track, 8.5ms solution per image at shape (1, 3, 1800, 3200)

49: 1800x3200 12.0ms
Speed: 260.0ms track, 12.0ms solution per image at shape (1, 3, 1800, 3200)

50: 1800x3200 12.0ms
Speed: 248.0ms track, 12.0ms solution per image at shape (1, 3, 1800, 3200)

51: 1800x3200 8.9ms
Speed: 257.9ms track, 8.9ms solution per image at shape (1, 3, 1800, 3200)

52: 1800x3200 8.1ms
Speed: 245.1ms track, 8.1ms solution per image at shape (1, 3, 1800, 3200)

53: 1800x3200 11.1ms
Speed: 259.9ms track, 11.1ms solution per image at shape (1, 3, 1800, 3200)

54: 1800x3200 6.9ms
Speed: 244.4ms track, 6.9ms solution per image at shape (1, 3, 1800, 3200)

55: 1800x3200 16.0ms
Speed: 260.8ms track, 16.0ms solution per image at shape (1, 3, 1800, 3200)

56: 1800x3200 9.0ms
Speed: 247.2ms track, 9.0ms solution per image at shape (1, 3, 1800, 3200)

57: 1800x3200 9.5ms
Speed: 260.6ms track, 9.5ms solution per image at shape (1, 3, 1800, 3200)

58: 1800x3200 6.6ms
Speed: 245.0ms track, 6.6ms solution per image at shape (1, 3, 1800, 3200)

59: 1800x3200 7.3ms
Speed: 257.8ms track, 7.3ms solution per image at shape (1, 3, 1800, 3200)

60: 1800x3200 5.3ms
Speed: 251.1ms track, 5.3ms solution per image at shape (1, 3, 1800, 3200)

61: 1800x3200 11.3ms
Speed: 279.4ms track, 11.3ms solution per image at shape (1, 3, 1800, 3200)

62: 1800x3200 11.1ms
Speed: 266.4ms track, 11.1ms solution per image at shape (1, 3, 1800, 3200)

63: 1800x3200 6.5ms
Speed: 274.8ms track, 6.5ms solution per image at shape (1, 3, 1800, 3200)

64: 1800x3200 12.3ms
Speed: 264.9ms track, 12.3ms solution per image at shape (1, 3, 1800, 3200)

65: 1800x3200 4.0ms
Speed: 258.8ms track, 4.0ms solution per image at shape (1, 3, 1800, 3200)

66: 1800x3200 4.2ms
Speed: 263.6ms track, 4.2ms solution per image at shape (1, 3, 1800, 3200)

67: 1800x3200 4.9ms
Speed: 256.6ms track, 4.9ms solution per image at shape (1, 3, 1800, 3200)

68: 1800x3200 6.5ms
Speed: 256.2ms track, 6.5ms solution per image at shape (1, 3, 1800, 3200)

69: 1800x3200 9.4ms
Speed: 254.6ms track, 9.4ms solution per image at shape (1, 3, 1800, 3200)

70: 1800x3200 7.7ms
Speed: 252.7ms track, 7.7ms solution per image at shape (1, 3, 1800, 3200)

71: 1800x3200 7.1ms
Speed: 249.9ms track, 7.1ms solution per image at shape (1, 3, 1800, 3200)

72: 1800x3200 6.1ms
Speed: 251.9ms track, 6.1ms solution per image at shape (1, 3, 1800, 3200)

73: 1800x3200 12.1ms
Speed: 250.1ms track, 12.1ms solution per image at shape (1, 3, 1800, 3200)

74: 1800x3200 7.7ms
Speed: 254.9ms track, 7.7ms solution per image at shape (1, 3, 1800, 3200)

75: 1800x3200 10.8ms
Speed: 253.4ms track, 10.8ms solution per image at shape (1, 3, 1800, 3200)

76: 1800x3200 14.7ms
Speed: 273.9ms track, 14.7ms solution per image at shape (1, 3, 1800, 3200)

77: 1800x3200 9.7ms
Speed: 259.6ms track, 9.7ms solution per image at shape (1, 3, 1800, 3200)

78: 1800x3200 9.4ms
Speed: 247.9ms track, 9.4ms solution per image at shape (1, 3, 1800, 3200)

79: 1800x3200 10.6ms
Speed: 261.0ms track, 10.6ms solution per image at shape (1, 3, 1800, 3200)

80: 1800x3200 4.2ms
Speed: 250.5ms track, 4.2ms solution per image at shape (1, 3, 1800, 3200)

81: 1800x3200 4.7ms
Speed: 251.7ms track, 4.7ms solution per image at shape (1, 3, 1800, 3200)

82: 1800x3200 5.4ms
Speed: 244.5ms track, 5.4ms solution per image at shape (1, 3, 1800, 3200)

83: 1800x3200 4.2ms
Speed: 252.6ms track, 4.2ms solution per image at shape (1, 3, 1800, 3200)

84: 1800x3200 6.8ms
Speed: 247.5ms track, 6.8ms solution per image at shape (1, 3, 1800, 3200)

85: 1800x3200 5.0ms
Speed: 251.8ms track, 5.0ms solution per image at shape (1, 3, 1800, 3200)

86: 1800x3200 4.3ms
Speed: 243.0ms track, 4.3ms solution per image at shape (1, 3, 1800, 3200)

87: 1800x3200 4.1ms
Speed: 253.6ms track, 4.1ms solution per image at shape (1, 3, 1800, 3200)

88: 1800x3200 5.0ms
Speed: 246.8ms track, 5.0ms solution per image at shape (1, 3, 1800, 3200)

89: 1800x3200 4.3ms
Speed: 264.1ms track, 4.3ms solution per image at shape (1, 3, 1800, 3200)

90: 1800x3200 3.3ms
Speed: 250.4ms track, 3.3ms solution per image at shape (1, 3, 1800, 3200)

91: 1800x3200 3.4ms
Speed: 262.2ms track, 3.4ms solution per image at shape (1, 3, 1800, 3200)

92: 1800x3200 5.9ms
Speed: 252.2ms track, 5.9ms solution per image at shape (1, 3, 1800, 3200)

93: 1800x3200 8.6ms
Speed: 253.2ms track, 8.6ms solution per image at shape (1, 3, 1800, 3200)

94: 1800x3200 3.3ms
Speed: 245.0ms track, 3.3ms solution per image at shape (1, 3, 1800, 3200)

95: 1800x3200 10.0ms
Speed: 255.6ms track, 10.0ms solution per image at shape (1, 3, 1800, 3200)

96: 1800x3200 8.4ms
Speed: 258.2ms track, 8.4ms solution per image at shape (1, 3, 1800, 3200)

97: 1800x3200 4.2ms
Speed: 250.3ms track, 4.2ms solution per image at shape (1, 3, 1800, 3200)

98: 1800x3200 2.8ms
Speed: 263.0ms track, 2.8ms solution per image at shape (1, 3, 1800, 3200)

99: 1800x3200 5.4ms
Speed: 257.5ms track, 5.4ms solution per image at shape (1, 3, 1800, 3200)

100: 1800x3200 4.7ms
Speed: 255.1ms track, 4.7ms solution per image at shape (1, 3, 1800, 3200)

101: 1800x3200 4.3ms
Speed: 250.5ms track, 4.3ms solution per image at shape (1, 3, 1800, 3200)

102: 1800x3200 4.3ms
Speed: 253.9ms track, 4.3ms solution per image at shape (1, 3, 1800, 3200)

103: 1800x3200 8.8ms
Speed: 251.4ms track, 8.8ms solution per image at shape (1, 3, 1800, 3200)

104: 1800x3200 6.0ms
Speed: 249.8ms track, 6.0ms solution per image at shape (1, 3, 1800, 3200)

105: 1800x3200 4.8ms
Speed: 242.5ms track, 4.8ms solution per image at shape (1, 3, 1800, 3200)

106: 1800x3200 7.0ms
Speed: 252.7ms track, 7.0ms solution per image at shape (1, 3, 1800, 3200)

107: 1800x3200 5.7ms
Speed: 243.0ms track, 5.7ms solution per image at shape (1, 3, 1800, 3200)

108: 1800x3200 4.1ms
Speed: 248.5ms track, 4.1ms solution per image at shape (1, 3, 1800, 3200)

109: 1800x3200 57.2ms
Speed: 240.0ms track, 57.2ms solution per image at shape (1, 3, 1800, 3200)

110: 1800x3200 4.6ms
Speed: 246.2ms track, 4.6ms solution per image at shape (1, 3, 1800, 3200)

111: 1800x3200 4.3ms
Speed: 237.7ms track, 4.3ms solution per image at shape (1, 3, 1800, 3200)

112: 1800x3200 3.6ms
Speed: 243.7ms track, 3.6ms solution per image at shape (1, 3, 1800, 3200)

113: 1800x3200 4.3ms
Speed: 237.8ms track, 4.3ms solution per image at shape (1, 3, 1800, 3200)

114: 1800x3200 4.1ms
Speed: 251.7ms track, 4.1ms solution per image at shape (1, 3, 1800, 3200)

115: 1800x3200 6.6ms
Speed: 242.5ms track, 6.6ms solution per image at shape (1, 3, 1800, 3200)

116: 1800x3200 5.8ms
Speed: 254.2ms track, 5.8ms solution per image at shape (1, 3, 1800, 3200)

[hevc @ 0x55b0e232d900] The cu_qp_delta 50 is outside the valid range [-26, 25].
117: 1800x3200 6.3ms
Speed: 250.7ms track, 6.3ms solution per image at shape (1, 3, 1800, 3200)

[hevc @ 0x55b0e252fd40] Could not find ref with POC 0
118: 1800x3200 7.5ms
Speed: 249.4ms track, 7.5ms solution per image at shape (1, 3, 1800, 3200)

119: 1800x3200 6.8ms
Speed: 247.7ms track, 6.8ms solution per image at shape (1, 3, 1800, 3200)

120: 1800x3200 3.4ms
Speed: 245.0ms track, 3.4ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
121: 1800x3200 1.0ms
Speed: 415.9ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
122: 1800x3200 1.0ms
Speed: 295.9ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
123: 1800x3200 1.0ms
Speed: 261.5ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
124: 1800x3200 1.0ms
Speed: 286.5ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
125: 1800x3200 1.0ms
Speed: 257.7ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

126: 1800x3200 1.7ms
Speed: 264.7ms track, 1.7ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
127: 1800x3200 1.0ms
Speed: 256.1ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
128: 1800x3200 1.0ms
Speed: 274.5ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
129: 1800x3200 1.0ms
Speed: 260.0ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

130: 1800x3200 1.9ms
Speed: 269.5ms track, 1.9ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
131: 1800x3200 1.0ms
Speed: 247.2ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
132: 1800x3200 1.0ms
Speed: 262.4ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
133: 1800x3200 1.0ms
Speed: 254.9ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

134: 1800x3200 1.9ms
Speed: 259.8ms track, 1.9ms solution per image at shape (1, 3, 1800, 3200)

WARNING ⚠️ no tracks found!
135: 1800x3200 1.0ms
Speed: 248.3ms track, 1.0ms solution per image at shape (1, 3, 1800, 3200)

Video frame is empty or video processing has been successfully completed.
"""

In [36]:
df = create_dataframe_from_output(output)

visualize_fps_over_time(df)

Inference with ROI

In [37]:
output = """0: 1099x2251 2.4ms
Speed: 420.8ms track, 2.4ms solution per image at shape (1, 3, 1099, 2251)

1: 1099x2251 2.5ms
Speed: 198.5ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

2: 1099x2251 3.5ms
Speed: 218.6ms track, 3.5ms solution per image at shape (1, 3, 1099, 2251)

3: 1099x2251 4.0ms
Speed: 190.6ms track, 4.0ms solution per image at shape (1, 3, 1099, 2251)

4: 1099x2251 4.0ms
Speed: 198.4ms track, 4.0ms solution per image at shape (1, 3, 1099, 2251)

5: 1099x2251 4.1ms
Speed: 196.2ms track, 4.1ms solution per image at shape (1, 3, 1099, 2251)

6: 1099x2251 3.2ms
Speed: 198.2ms track, 3.2ms solution per image at shape (1, 3, 1099, 2251)

7: 1099x2251 3.4ms
Speed: 195.9ms track, 3.4ms solution per image at shape (1, 3, 1099, 2251)

8: 1099x2251 3.6ms
Speed: 199.6ms track, 3.6ms solution per image at shape (1, 3, 1099, 2251)

9: 1099x2251 3.5ms
Speed: 197.4ms track, 3.5ms solution per image at shape (1, 3, 1099, 2251)

10: 1099x2251 4.5ms
Speed: 193.9ms track, 4.5ms solution per image at shape (1, 3, 1099, 2251)

11: 1099x2251 4.6ms
Speed: 195.7ms track, 4.6ms solution per image at shape (1, 3, 1099, 2251)

12: 1099x2251 3.2ms
Speed: 197.2ms track, 3.2ms solution per image at shape (1, 3, 1099, 2251)

13: 1099x2251 3.2ms
Speed: 195.7ms track, 3.2ms solution per image at shape (1, 3, 1099, 2251)

14: 1099x2251 3.2ms
Speed: 202.6ms track, 3.2ms solution per image at shape (1, 3, 1099, 2251)

15: 1099x2251 3.3ms
Speed: 195.0ms track, 3.3ms solution per image at shape (1, 3, 1099, 2251)

16: 1099x2251 1.8ms
Speed: 204.9ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

17: 1099x2251 3.2ms
Speed: 194.3ms track, 3.2ms solution per image at shape (1, 3, 1099, 2251)

18: 1099x2251 1.8ms
Speed: 198.1ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
19: 1099x2251 1.0ms
Speed: 190.9ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

20: 1099x2251 1.9ms
Speed: 193.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

21: 1099x2251 2.7ms
Speed: 194.7ms track, 2.7ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
22: 1099x2251 1.0ms
Speed: 197.2ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

23: 1099x2251 1.8ms
Speed: 251.3ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
24: 1099x2251 1.1ms
Speed: 191.7ms track, 1.1ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
25: 1099x2251 1.0ms
Speed: 192.8ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
26: 1099x2251 1.0ms
Speed: 199.6ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
27: 1099x2251 1.0ms
Speed: 195.9ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
28: 1099x2251 1.0ms
Speed: 193.9ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
29: 1099x2251 1.0ms
Speed: 193.1ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
30: 1099x2251 1.0ms
Speed: 196.4ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
31: 1099x2251 1.0ms
Speed: 196.7ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
32: 1099x2251 1.0ms
Speed: 197.3ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
33: 1099x2251 1.0ms
Speed: 194.8ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
34: 1099x2251 1.0ms
Speed: 191.7ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

35: 1099x2251 1.6ms
Speed: 199.2ms track, 1.6ms solution per image at shape (1, 3, 1099, 2251)

36: 1099x2251 1.9ms
Speed: 192.8ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

37: 1099x2251 1.8ms
Speed: 196.6ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

38: 1099x2251 1.9ms
Speed: 197.6ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

39: 1099x2251 1.9ms
Speed: 194.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

40: 1099x2251 1.8ms
Speed: 202.5ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

41: 1099x2251 1.8ms
Speed: 206.0ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

42: 1099x2251 1.9ms
Speed: 200.6ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

43: 1099x2251 1.9ms
Speed: 193.8ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

44: 1099x2251 1.8ms
Speed: 208.2ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

45: 1099x2251 1.9ms
Speed: 195.8ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

46: 1099x2251 1.9ms
Speed: 196.0ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

47: 1099x2251 1.8ms
Speed: 193.5ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

48: 1099x2251 1.9ms
Speed: 195.0ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

49: 1099x2251 1.9ms
Speed: 200.1ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

50: 1099x2251 1.9ms
Speed: 194.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

51: 1099x2251 1.9ms
Speed: 194.2ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

52: 1099x2251 1.9ms
Speed: 197.0ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

53: 1099x2251 1.9ms
Speed: 194.1ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

54: 1099x2251 1.9ms
Speed: 194.3ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

55: 1099x2251 2.0ms
Speed: 189.8ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

56: 1099x2251 1.9ms
Speed: 194.2ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

57: 1099x2251 1.9ms
Speed: 189.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

58: 1099x2251 2.4ms
Speed: 203.7ms track, 2.4ms solution per image at shape (1, 3, 1099, 2251)

59: 1099x2251 2.0ms
Speed: 189.2ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

60: 1099x2251 1.8ms
Speed: 193.4ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

61: 1099x2251 1.7ms
Speed: 193.8ms track, 1.7ms solution per image at shape (1, 3, 1099, 2251)

62: 1099x2251 2.2ms
Speed: 198.3ms track, 2.2ms solution per image at shape (1, 3, 1099, 2251)

63: 1099x2251 3.3ms
Speed: 187.0ms track, 3.3ms solution per image at shape (1, 3, 1099, 2251)

64: 1099x2251 2.5ms
Speed: 193.8ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

65: 1099x2251 2.4ms
Speed: 191.7ms track, 2.4ms solution per image at shape (1, 3, 1099, 2251)

66: 1099x2251 2.5ms
Speed: 200.1ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

67: 1099x2251 2.5ms
Speed: 189.1ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

68: 1099x2251 1.8ms
Speed: 192.7ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

69: 1099x2251 2.5ms
Speed: 194.6ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
70: 1099x2251 1.0ms
Speed: 191.2ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

71: 1099x2251 2.4ms
Speed: 194.8ms track, 2.4ms solution per image at shape (1, 3, 1099, 2251)

72: 1099x2251 2.4ms
Speed: 193.2ms track, 2.4ms solution per image at shape (1, 3, 1099, 2251)

73: 1099x2251 2.5ms
Speed: 192.7ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

74: 1099x2251 2.5ms
Speed: 195.4ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

75: 1099x2251 1.8ms
Speed: 193.1ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

76: 1099x2251 2.5ms
Speed: 199.9ms track, 2.5ms solution per image at shape (1, 3, 1099, 2251)

77: 1099x2251 2.6ms
Speed: 191.9ms track, 2.6ms solution per image at shape (1, 3, 1099, 2251)

78: 1099x2251 1.8ms
Speed: 193.7ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

79: 1099x2251 1.9ms
Speed: 188.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

80: 1099x2251 1.8ms
Speed: 196.4ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
81: 1099x2251 1.0ms
Speed: 187.5ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
82: 1099x2251 1.1ms
Speed: 195.5ms track, 1.1ms solution per image at shape (1, 3, 1099, 2251)

83: 1099x2251 2.3ms
Speed: 192.0ms track, 2.3ms solution per image at shape (1, 3, 1099, 2251)

84: 1099x2251 2.4ms
Speed: 191.1ms track, 2.4ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
85: 1099x2251 1.0ms
Speed: 187.6ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

86: 1099x2251 1.8ms
Speed: 195.1ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

87: 1099x2251 2.2ms
Speed: 187.2ms track, 2.2ms solution per image at shape (1, 3, 1099, 2251)

88: 1099x2251 1.9ms
Speed: 189.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

89: 1099x2251 1.9ms
Speed: 190.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

90: 1099x2251 1.8ms
Speed: 196.3ms track, 1.8ms solution per image at shape (1, 3, 1099, 2251)

91: 1099x2251 1.9ms
Speed: 193.2ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

92: 1099x2251 1.9ms
Speed: 195.1ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

93: 1099x2251 1.9ms
Speed: 194.8ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

94: 1099x2251 1.9ms
Speed: 193.5ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

95: 1099x2251 2.0ms
Speed: 189.0ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

96: 1099x2251 1.9ms
Speed: 192.1ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

97: 1099x2251 3.1ms
Speed: 191.7ms track, 3.1ms solution per image at shape (1, 3, 1099, 2251)

98: 1099x2251 2.8ms
Speed: 193.0ms track, 2.8ms solution per image at shape (1, 3, 1099, 2251)

99: 1099x2251 2.0ms
Speed: 189.5ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

100: 1099x2251 3.0ms
Speed: 195.2ms track, 3.0ms solution per image at shape (1, 3, 1099, 2251)

101: 1099x2251 2.0ms
Speed: 196.9ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

102: 1099x2251 2.6ms
Speed: 198.8ms track, 2.6ms solution per image at shape (1, 3, 1099, 2251)

103: 1099x2251 2.8ms
Speed: 189.0ms track, 2.8ms solution per image at shape (1, 3, 1099, 2251)

104: 1099x2251 2.6ms
Speed: 194.2ms track, 2.6ms solution per image at shape (1, 3, 1099, 2251)

105: 1099x2251 2.6ms
Speed: 196.0ms track, 2.6ms solution per image at shape (1, 3, 1099, 2251)

106: 1099x2251 2.0ms
Speed: 197.3ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

107: 1099x2251 2.0ms
Speed: 190.7ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

108: 1099x2251 2.0ms
Speed: 198.0ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

109: 1099x2251 2.0ms
Speed: 199.2ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

110: 1099x2251 2.1ms
Speed: 202.0ms track, 2.1ms solution per image at shape (1, 3, 1099, 2251)

111: 1099x2251 2.1ms
Speed: 199.7ms track, 2.1ms solution per image at shape (1, 3, 1099, 2251)

112: 1099x2251 2.0ms
Speed: 197.4ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

113: 1099x2251 2.0ms
Speed: 199.8ms track, 2.0ms solution per image at shape (1, 3, 1099, 2251)

114: 1099x2251 2.1ms
Speed: 200.0ms track, 2.1ms solution per image at shape (1, 3, 1099, 2251)

115: 1099x2251 2.1ms
Speed: 199.3ms track, 2.1ms solution per image at shape (1, 3, 1099, 2251)

116: 1099x2251 2.1ms
Speed: 192.6ms track, 2.1ms solution per image at shape (1, 3, 1099, 2251)

[hevc @ 0x55d29995d5c0] The cu_qp_delta 50 is outside the valid range [-26, 25].
117: 1099x2251 2.2ms
Speed: 196.7ms track, 2.2ms solution per image at shape (1, 3, 1099, 2251)

[hevc @ 0x55d299b5f780] Could not find ref with POC 0
118: 1099x2251 2.3ms
Speed: 195.7ms track, 2.3ms solution per image at shape (1, 3, 1099, 2251)

119: 1099x2251 2.3ms
Speed: 192.4ms track, 2.3ms solution per image at shape (1, 3, 1099, 2251)

120: 1099x2251 2.1ms
Speed: 198.2ms track, 2.1ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
121: 1099x2251 1.0ms
Speed: 270.4ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

122: 1099x2251 1.5ms
Speed: 234.9ms track, 1.5ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
123: 1099x2251 1.1ms
Speed: 230.6ms track, 1.1ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
124: 1099x2251 1.0ms
Speed: 240.2ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
125: 1099x2251 1.0ms
Speed: 210.0ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

126: 1099x2251 1.6ms
Speed: 209.8ms track, 1.6ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
127: 1099x2251 1.0ms
Speed: 203.0ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
128: 1099x2251 1.0ms
Speed: 188.9ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
129: 1099x2251 1.0ms
Speed: 207.3ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

130: 1099x2251 1.9ms
Speed: 198.7ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

131: 1099x2251 1.9ms
Speed: 198.2ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

132: 1099x2251 1.9ms
Speed: 212.4ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

133: 1099x2251 1.9ms
Speed: 205.9ms track, 1.9ms solution per image at shape (1, 3, 1099, 2251)

WARNING ⚠️ no tracks found!
134: 1099x2251 1.0ms
Speed: 197.7ms track, 1.0ms solution per image at shape (1, 3, 1099, 2251)

135: 1099x2251 1.5ms
Speed: 190.4ms track, 1.5ms solution per image at shape (1, 3, 1099, 2251)

Video frame is empty or video processing has been successfully completed.
"""

In [38]:
df = create_dataframe_from_output(output)

visualize_fps_over_time(df)